# Customer Churn Prediction Project

This project predicts customer churn for a fictional telecommunications company. By analyzing customer data, a Logistic Regression model was developed to identify customers likely to cancel their subscriptions, achieving an accuracy of approximately 81.3%.

In [4]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
import joblib

## 1. Data Loading and Exploration

In [5]:
# Load the dataset
churn_df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

# Display the first 5 rows
churn_df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [6]:
# Get a summary of the DataFrame
churn_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


## 2. Data Preprocessing and Cleaning

In [7]:
# Fix 'TotalCharges' column and drop rows with missing values
churn_df['TotalCharges'] = pd.to_numeric(churn_df['TotalCharges'], errors='coerce')
churn_df.dropna(inplace=True)

# Encode binary and target variables
churn_df['Churn'] = churn_df['Churn'].map({'No': 0, 'Yes': 1})
churn_df['gender'] = churn_df['gender'].map({'Female': 0, 'Male': 1})
binary_cols = ['Partner', 'Dependents', 'PhoneService', 'PaperlessBilling']
for col in binary_cols:
    churn_df[col] = churn_df[col].map({'No': 0, 'Yes': 1})

# One-Hot Encode multi-category columns and drop customerID
multi_cat_cols = churn_df.select_dtypes(include=['object']).columns.drop(['customerID'])
churn_df_final = pd.get_dummies(churn_df, columns=multi_cat_cols, drop_first=True)
churn_df_final = churn_df_final.drop('customerID', axis=1)

### Final Data Check
As a final preprocessing step, we clean the column names for API compatibility and check the data types to ensure everything is numeric.

In [8]:
# Clean column names
churn_df_final.columns = churn_df_final.columns.str.replace(' ', '_')

# Verify that all columns are now numeric
print("Final Data Types:")
churn_df_final.info()

Final Data Types:
<class 'pandas.core.frame.DataFrame'>
Index: 7032 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   gender                                 7032 non-null   int64  
 1   SeniorCitizen                          7032 non-null   int64  
 2   Partner                                7032 non-null   int64  
 3   Dependents                             7032 non-null   int64  
 4   tenure                                 7032 non-null   int64  
 5   PhoneService                           7032 non-null   int64  
 6   PaperlessBilling                       7032 non-null   int64  
 7   MonthlyCharges                         7032 non-null   float64
 8   TotalCharges                           7032 non-null   float64
 9   Churn                                  7032 non-null   int64  
 10  MultipleLines_No_phone_service         7032 non-null   bool

## 3. Model Building and Training

In [9]:
# Separate features (X) and target (y)
X = churn_df_final.drop('Churn', axis=1)
y = churn_df_final['Churn']

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Initialize and train the Logistic Regression model
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train_scaled, y_train)

LogisticRegression(max_iter=1000)

## 4. Model Evaluation

In [10]:
# Evaluate the model's accuracy
accuracy = log_model.score(X_test_scaled, y_test)
print(f"Model Accuracy: {accuracy:.4f}")

# Generate the confusion matrix
y_pred = log_model.predict(X_test_scaled)
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:")
print(cm)

Model Accuracy: 0.7901

Confusion Matrix:
[[1152  148]
 [ 221  237]]


## 5. Save Model and Scaler
The final trained model and the scaler are saved to disk for use in the API.

In [11]:
# Save the model and scaler to disk
joblib.dump(log_model, 'churn_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

print("Model and Scaler saved successfully.")

Model and Scaler saved successfully.
